In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/disease_detection


Mounted at /content/drive
/content/drive/MyDrive/disease_detection


In [ ]:
#!git clone https://github.com/lxfhfut/Self-Supervised-Leaf-Segmentation.git

^C


In [ ]:
import torch
!pip install imutils
!pip install patchify
!pip install fastai
%cd Self_Supervised_Leaf_Segmentation

/content/drive/MyDrive/disease_detection/Self_Supervised_Leaf_Segmentation


In [ ]:
!python leaf_segmenter.py --input ./../Data/data_masked/test/scap/2image.png #\
                          #--save_video True


Iterations 199/200: 13, 3.20: 100% 200/200 [00:33<00:00,  5.90it/s]
Figure(1500x1000)
Result has been saved in ./output/result.jpg
Iterations 199/200: 18, 3.20: 100% 200/200 [00:31<00:00,  6.28it/s]
Figure(1500x1000)
Result has been saved in ./output/result.jpg
Iterations 199/200: 16, 3.20: 100% 200/200 [00:31<00:00,  6.30it/s]
Figure(1500x1000)
Result has been saved in ./output/result.jpg
Iterations 199/200: 8, 3.21: 100% 200/200 [00:31<00:00,  6.29it/s]
Figure(1500x1000)
Result has been saved in ./output/result.jpg
Iterations 199/200: 9, 3.20: 100% 200/200 [00:31<00:00,  6.30it/s]
Figure(1500x1000)
Result has been saved in ./output/result.jpg
Iterations 199/200: 13, 3.20: 100% 200/200 [00:31<00:00,  6.30it/s]
Figure(1500x1000)
Result has been saved in ./output/result.jpg
Iterations 199/200: 14, 3.20: 100% 200/200 [00:31<00:00,  6.30it/s]
Figure(1500x1000)
Result has been saved in ./output/result.jpg
Iterations 199/200: 11, 3.20: 100% 200/200 [00:31<00:00,  6.29it/s]
Figure(1500x1000)

In [ ]:
def save_image(image, image_name, count, Method, specific_folder, category):
    """Save an image to the specified path."""
    cv2.imwrite('./../comparision_leaf_segmentation/'+ str(Method)+ '/' + str(specific_folder)+ '/' + str(category)+ '/' + str(count) + '_' + str(image_name)+ '.png', image)
import cv2
import numpy as np


def calculate_dice_coefficient(target, prediction):
    intersection = np.logical_and(target, prediction)
    dice = (2. * np.sum(intersection)) / (np.sum(target) + np.sum(prediction))
    return dice

def color_difference(image_path1, groundtruth_path, dice_score_dict, count):
    # Read the images
    image1 = cv2.imread(image_path1, cv2.IMREAD_GRAYSCALE)  # Load in grayscale
    groundtruth = cv2.imread(groundtruth_path, cv2.IMREAD_GRAYSCALE)  # Load in grayscale

    if image1 is None or groundtruth is None:
        raise ValueError("Could not read one of the images!")

    diff1 = cv2.subtract(image1, groundtruth)  # Areas unique to image1
    diff2 = cv2.subtract(groundtruth, image1)  # Areas unique to image2

    # Find common areas
    common = cv2.bitwise_and(image1, groundtruth)

    # Threshold the differences and common areas
    _, thresh_diff1 = cv2.threshold(diff1, 30, 255, cv2.THRESH_BINARY)
    _, thresh_diff2 = cv2.threshold(diff2, 30, 255, cv2.THRESH_BINARY)
    _, thresh_common = cv2.threshold(common, 30, 255, cv2.THRESH_BINARY)

    # Prepare a color image for result
    superimposed = np.zeros((*image1.shape, 3), dtype=np.uint8)  # Same size as input, but 3 channels

    # Set colors: unique parts of image1 as green, image2 as red, and overlap as white
    superimposed[thresh_diff1 == 255] = (0, 255, 0)  # green
    superimposed[thresh_diff2 == 255] = (0, 0, 255)  # red
    superimposed[thresh_common == 255] = (255, 255, 255)  # white
    #generating green and red versions
    lower_white = np.array([230])  # Slightly less than pure white to capture near white
    upper_white = np.array([255])

    # Create masks to detect all white pixels
    white_pixels1 = cv2.inRange(image1, lower_white, upper_white)
    white_pixels2 = cv2.inRange(groundtruth, lower_white, upper_white)

    groundtruth_green = np.zeros((*image1.shape, 3), dtype=np.uint8)  # Same size as input, but 3 channels
    superimposed2 = np.zeros((*image1.shape, 3), dtype=np.uint8)  # Same size as input, but 3 channels

    # Change all white pixels to red
    groundtruth_green[white_pixels1 == 255] = [0, 255, 0]
    superimposed2[white_pixels2 == 255] = [0, 0, 255]
    image1 = image1/255.
    image1[image1 > 0.5] = 1
    image1[image1 <= 0.5] = 0
    groundtruth = groundtruth/255.
    groundtruth[groundtruth > 0.5] = 1
    groundtruth[groundtruth <= 0.5] = 0

    #_, image1 = cv2.threshold(image1, 127, 255, cv2.THRESH_BINARY)
    #image1 = np.where(image1 == 255, 1, 0)
    #_, image2 = cv2.threshold(image2, 127, 255, cv2.THRESH_BINARY)
    #image2 = np.where(image2 == 255, 1, 0)
    dice_score_dict[count] = calculate_dice_coefficient(groundtruth, image1)

    return groundtruth_green, superimposed
#Replace 'path_to_image1.jpg' and 'path_to_image2.jpg' with your actual file paths
def masking_OpenCV(img_path):
        img = cv2.imread(img_path)
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)    # Change the colorspace
        a_channel = lab[:, :, 1]                        # Select green color channel
        th = cv2.threshold(a_channel, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1] # Threshold with OTSU
        th = cv2.dilate(th, None, iterations=7) # Bring Contours togesther
        th = cv2.erode(th, None, iterations=8)
        th = th.astype(np.uint8)
        #cv2.imwrite('./results_all/'+'data_masked'+'/'+str(count)+'_'+'th.png', th)
        contours = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = imutils.grab_contours(contours)
        # Create a mask to fill the area around the contours
        mask = img.copy()
        for cnt in contours:
            cv2.fillPoly(mask, [cnt]
                        , (0, 0, 0))  # Fill the area around the contours with black
        # Create an inverse mask to keep the original content within the contours
        inverse_mask = cv2.bitwise_not(mask)
        # Combine the original image with the filled area using the inverse mask
        img = cv2.bitwise_and(img, inverse_mask)
        return th, img


import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    A list of integer or text chunks to be used as keys for sorting.
    "z23.txt" -> ["z", 23, ".txt"]
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text)]

In [ ]:
import os
import imutils
#generating mask with openCV
input_folder = "./../Data/data_masked/test/good/"
input_files = os.listdir(input_folder)
input_files.sort(key=natural_keys)
count =  0
for filename in input_files:
  if filename.endswith(('.png')):
    print(filename)
    pth_to_img = os.path.join(input_folder, filename)
    th, _ =  masking_2(pth_to_img)
    save_image(image = th , image_name = 'th', count = count, Method= 'OpenCV', specific_folder= 'generated_masks',category='good')
    count += 1

NameError: ignored

In [ ]:
# @title Comparision OpenCV
import pandas as pd
import os
import imutils
#generating mask with openCV
input_folder = "./../Data/data_masked/test/good/"
input_files = os.listdir(input_folder)
input_files.sort(key=natural_keys)
count =  0
for filename in input_files:
  if filename.endswith(('.png')):
    print(filename)
    pth_to_img = os.path.join(input_folder, filename)
    th, _ =  masking_OpenCV(pth_to_img)
    save_image(image = th , image_name = 'th_good', count = count, Method= 'OpenCV', specific_folder= 'generated_masks',category='good')
    count += 1
#generating mask with openCV
input_folder = "./../Data/data_masked/test/scap/"
input_files = os.listdir(input_folder)
input_files.sort(key=natural_keys)
count =  0
for filename in input_files:
  if filename.endswith(('.png')):
    print(filename)
    pth_to_img = os.path.join(input_folder, filename)
    th, _ =  masking_OpenCV(pth_to_img)
    save_image(image = th , image_name = 'th_scap', count = count, Method= 'OpenCV', specific_folder= 'generated_masks',category='scap')
    count += 1
#generating comparisions between groundtruth and OpenCV for good images
input_folder_openCV = "./../comparision_leaf_segmentation/OpenCV/generated_masks/good/"
input_files_openCV = os.listdir(input_folder_openCV)
input_files_openCV.sort(key=natural_keys)

input_folder_groundtruth = "./../Data/data_masked/ground_truth_leaf/good/"
input_files_groundtruth = os.listdir(input_folder_groundtruth)
input_files_groundtruth.sort(key=natural_keys)
dice_score_dict, iou_score_dict = {},{}
count =  0
for file_openCV, file_groundtruth in zip(input_files_openCV, input_files_groundtruth):
    print(f"Processing {file_openCV} and {file_groundtruth}")  # Check if loop is entered

    pth_to_file_openCV = os.path.join(input_folder_openCV, file_openCV)
    pth_to_file_groundtruth = os.path.join(input_folder_groundtruth, file_groundtruth)
    groundtruth_green, superimposed  = color_difference(pth_to_file_openCV, pth_to_file_groundtruth, dice_score_dict,  count)
    save_image(image = superimposed , image_name = 'superimposed_OpenCV_good', count = count, Method= 'OpenCV', specific_folder= 'comparision_files',category='good')
    save_image(image = groundtruth_green , image_name = 'groundtruth_green_good', count = count, Method= 'green_ground_truth', specific_folder= 'specific_folder', category='good')
    count += 1
dice_score_dict = pd.DataFrame(dice_score_dict.values(), columns=['dice_score'], index=dice_score_dict.keys())
with open("./../comparision_leaf_segmentation/OpenCV/comparision_files/good/dice_score_OpenCV_good.txt", "w") as f:
             f.write(dice_score_dict.to_string())
#generating comparisions between groundtruth and OpenCV for scap images
input_folder_openCV = "./../comparision_leaf_segmentation/OpenCV/generated_masks/scap/"
input_files_openCV = os.listdir(input_folder_openCV)
input_files_openCV.sort(key=natural_keys)

input_folder_groundtruth = "./../Data/data_masked/ground_truth_leaf/scap/"
input_files_groundtruth = os.listdir(input_folder_groundtruth)
input_files_groundtruth.sort(key=natural_keys)
dice_score_dict, iou_score_dict = {},{}
count =  0
for file_openCV, file_groundtruth in zip(input_files_openCV, input_files_groundtruth):
    print(f"Processing {file_openCV} and {file_groundtruth}")  # Check if loop is entered

    pth_to_file_openCV = os.path.join(input_folder_openCV, file_openCV)
    pth_to_file_groundtruth = os.path.join(input_folder_groundtruth, file_groundtruth)
    groundtruth_green, superimposed  = color_difference(pth_to_file_openCV, pth_to_file_groundtruth, dice_score_dict,  count)
    save_image(image = superimposed , image_name = 'superimposed_OpenCV_scap', count = count, Method= 'OpenCV', specific_folder= 'comparision_files',category='scap')
    save_image(image = groundtruth_green , image_name = 'groundtruth_green_scap', count = count, Method= 'green_ground_truth', specific_folder= 'specific_folder', category='scap')
    count += 1
dice_score_dict = pd.DataFrame(dice_score_dict.values(), columns=['dice_score'], index=dice_score_dict.keys())
with open("./../comparision_leaf_segmentation/OpenCV/comparision_files/scap/dice_score_OpenCV_scap.txt", "w") as f:
             f.write(dice_score_dict.to_string())


0image.png
1image.png
2image.png
3image.png
4image.png
5image.png
6image.png
7image.png
8image.png
0image.png
1image.png
2image.png
3image.png
4image.png
5image.png
6image.png
7image.png
8image.png
9image.png
10image.png
11image.png
12image.png
13image.png
14image.png
15image.png
16image.png
17image.png
18image.png
19image.png
20image.png
21image.png
22image.png
23image.png
24image.png
25image.png
26image.png
27image.png
28image.png
29image.png
30image.png
31image.png
32image.png
Processing 0_th_good.png and 0leaf_mask.png
Processing 1_th_good.png and 1leaf_mask.png
Processing 2_th_good.png and 2leaf_mask.png
Processing 3_th_good.png and 3leaf_mask.png
Processing 4_th_good.png and 4leaf_mask.png
Processing 5_th_good.png and 5leaf_mask.png
Processing 6_th_good.png and 6leaf_mask.png
Processing 7_th_good.png and 7leaf_mask.png
Processing 8_th_good.png and 8leaf_mask.png
Processing 0_th_scap.png and 0leaf_mask.png
Processing 1_th_scap.png and 1leaf_mask.png
Processing 2_th_scap.png and 2l

In [ ]:
# @title Comparision Instand Segmentation
import pandas as pd
import os
import imutils
#generating mask with Instand Segmentation
!python leaf_segmenter.py --input ./../Data/data_masked/test/scap/2image.png                                                     --input_folder ./../Data/data_masked/test/good/ \
                          --input_folder ./../Data/data_masked/test/good/ \
                          --comparison_folder ./../comparision_leaf_segmentation/Self_supervised_segmentation/generated_masks/good/
!python leaf_segmenter.py --input ./../Data/data_masked/test/scap/2image.png \
                          --input_folder ./../Data/data_masked/test/scap/ \
                          --comparison_folder ./../comparision_leaf_segmentation/Self_supervised_segmentation/generated_masks/scap/

#generating comparisions between groundtruth and OpenCV for good images
input_folder_openCV = "./../comparision_leaf_segmentation/Self_supervised_segmentation/generated_masks/good/"
input_files_openCV = os.listdir(input_folder_openCV)
input_files_openCV.sort(key=natural_keys)

input_folder_groundtruth = "./../Data/data_masked/ground_truth_leaf/good/"
input_files_groundtruth = os.listdir(input_folder_groundtruth)
input_files_groundtruth.sort(key=natural_keys)
dice_score_dict, iou_score_dict = {},{}
count =  0
for file_openCV, file_groundtruth in zip(input_files_openCV, input_files_groundtruth):
    print(f"Processing {file_openCV} and {file_groundtruth}")  # Check if loop is entered

    pth_to_file_openCV = os.path.join(input_folder_openCV, file_openCV)
    pth_to_file_groundtruth = os.path.join(input_folder_groundtruth, file_groundtruth)
    groundtruth_green, superimposed  = color_difference(pth_to_file_openCV, pth_to_file_groundtruth, dice_score_dict,  count)
    save_image(image = superimposed , image_name = 'superimposed_Self_supervised_segmentation_good', count = count, Method= 'Self_supervised_segmentation', specific_folder= 'comparision_files',category='good')
    #save_image(image = groundtruth_green , image_name = 'groundtruth_green', count = count, Method= 'green_ground_truth', specific_folder= 'specific_folder', category='good')
    count += 1
dice_score_dict = pd.DataFrame(dice_score_dict.values(), columns=['dice_score'], index=dice_score_dict.keys())
with open("./../comparision_leaf_segmentation/Self_supervised_segmentation/comparision_files/good/dice_score_Self_supervised_segmentation_good.txt", "w") as f:
             f.write(dice_score_dict.to_string())
#generating comparisions between groundtruth and OpenCV for scap images
input_folder_openCV = "./../comparision_leaf_segmentation/Self_supervised_segmentation/generated_masks/scap/"
input_files_openCV = os.listdir(input_folder_openCV)
input_files_openCV.sort(key=natural_keys)

input_folder_groundtruth = "./../Data/data_masked/ground_truth_leaf/scap/"
input_files_groundtruth = os.listdir(input_folder_groundtruth)
input_files_groundtruth.sort(key=natural_keys)
dice_score_dict, iou_score_dict = {},{}
count =  0
for file_openCV, file_groundtruth in zip(input_files_openCV, input_files_groundtruth):
    print(f"Processing {file_openCV} and {file_groundtruth}")  # Check if loop is entered

    pth_to_file_openCV = os.path.join(input_folder_openCV, file_openCV)
    pth_to_file_groundtruth = os.path.join(input_folder_groundtruth, file_groundtruth)
    groundtruth_green, superimposed  = color_difference(pth_to_file_openCV, pth_to_file_groundtruth, dice_score_dict,  count)
    save_image(image = superimposed , image_name = 'superimposed_Self_supervised_segmentation_scap', count = count, Method= 'Self_supervised_segmentation', specific_folder= 'comparision_files',category='scap')
    #save_image(image = groundtruth_green , image_name = 'groundtruth_green', count = count, Method= 'green_ground_truth', specific_folder= 'specific_folder', category='scap')
    count += 1
dice_score_dict = pd.DataFrame(dice_score_dict.values(), columns=['dice_score'], index=dice_score_dict.keys())
with open("./../comparision_leaf_segmentation/Self_supervised_segmentation/comparision_files/scap/dice_score_Self_supervised_segmentation_scap.txt", "w") as f:
             f.write(dice_score_dict.to_string())


Iterations 199/200: 15, 3.20: 100% 200/200 [00:17<00:00, 11.13it/s]
Figure(1500x1000)
Result has been saved in ./output/result.jpg
Iterations 199/200: 17, 3.20: 100% 200/200 [00:10<00:00, 19.32it/s]
Figure(1500x1000)
Result has been saved in ./output/result.jpg
Iterations 199/200: 16, 3.20: 100% 200/200 [00:10<00:00, 19.30it/s]
Figure(1500x1000)
Result has been saved in ./output/result.jpg
Iterations 199/200: 7, 3.20: 100% 200/200 [00:10<00:00, 19.18it/s]
Figure(1500x1000)
Result has been saved in ./output/result.jpg
Iterations 199/200: 9, 3.20: 100% 200/200 [00:10<00:00, 19.28it/s]
Figure(1500x1000)
Result has been saved in ./output/result.jpg
Iterations 199/200: 12, 3.20: 100% 200/200 [00:10<00:00, 19.34it/s]
Figure(1500x1000)
Result has been saved in ./output/result.jpg
Iterations 199/200: 14, 3.20: 100% 200/200 [00:10<00:00, 19.19it/s]
Figure(1500x1000)
Result has been saved in ./output/result.jpg
Iterations 199/200: 11, 3.20: 100% 200/200 [00:10<00:00, 19.22it/s]
Figure(1500x1000)